In [ ]:
# VEEP - EVERY FRAME IN ORDER BOT
# Standard Nguyen - August 4st, 2024
# Based on the TWWbot by @laserdyke on Github

import os
import time
import sys

import tweepy
from dotenv import load_dotenv
from showenv import destination_directory as frame_dir
from get_tweet_data import get_tweet_data

In [ ]:
# Config and Env

#env
load_dotenv()
consumer_key = os.getenv('CONSUMER_KEY')
consumer_secret = os.getenv('CONSUMER_SECRET')
access_token = os.getenv('ACCESS_TOKEN')
access_token_secret = os.getenv('ACCESS_TOKEN_SECRET')

# Timing
wait_time = 1800 # Seconds after a Tweet, 30 minutes by default

In [ ]:
# Initialize the OAuth 1.0a for media upload using API v1.1
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create API object for v1.1 to handle media uploads
api = tweepy.API(auth)

# Initialize the Client object for API v2 for tweet creation
client = tweepy.Client(
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)

# try:
#     # Verify credentials with API v2
#     user = client.get_me()
#     print(f"Connected to Twitter using API v2 as: {user.data.username}")

#     # Path to the image file you want to upload
#     image_path = 'test.jpg'

#     # Upload the image using the v1.1 API
#     media = api.media_upload(image_path)
#     print("Image uploaded successfully!")

#     # Post a tweet with the uploaded image using the v2 API
#     tweet_text = "Test"
#     response = client.create_tweet(text=tweet_text, media_ids=[media.media_id])
#     print("Tweet with image posted successfully!", response.data)

# except tweepy.TweepyException as e:
#     print("Error:", e)
#     sys.exit()


In [ ]:
# while True:
#     # Check for JPEG files in the directory recursively
#     jpeg_found = False
#     for root, _, files in os.walk(frame_dir):
#         for file in files:
#             if file.lower().endswith(('.jpeg', '.jpg')):
#                 jpeg_found = True
#                 break
#         if jpeg_found:
#             break
    
#     if jpeg_found:
#         print("JPEG files found. Exiting the loop.")
#         break  # Exit the loop if any JPEG files are found
#     else:
#         print("No JPEG files found. Sleeping for 5 minutes.")
#         time.sleep(5 * 60)  # Sleep for 5 minutes

# # Wait for 10 minutes after exiting the loop
# print("Waiting for 10 minutes after exiting the loop.")
# time.sleep(10 * 60)  # Sleep for 10 minutes

In [ ]:
# Load the index from a file (or initialize it to 0 if the file doesn't exist)
index_file = "progress.txt"
if os.path.exists(index_file):
    with open(index_file) as f:
        frames_posted = int(f.read().strip())
    print("Progress.txt found. Resuming...")
else:
    frames_posted = 0
    print("No progress.txt found. From the top...")

In [ ]:
while True:
    (frame_filepath, tweet_text) = get_tweet_data(frame_dir, frames_posted)
    
    retries = 0
    success = False
    while not success and retries < 5:
        try:
            # Upload the images and get IDs
            media_ids = []
            media_metadata_list = []
            # Upload images
            response = api.media_upload(filename=frame_filepath)
            print(f"Uploaded {frame_filepath}")
            # Append media ID to list
            media_ids.append(response.media_id)
            # Tweet the images
            tweet = client.create_tweet(text=tweet_text, media_ids=media_ids)
            success = True
        except tweepy.errors.TweepyException as error:
            print(f'Error while tweeting images {frame_filepath}: {error}')
            retries += 1
            print('Trying again in 60.')
            time.sleep(60)
    if not success:
        print(f'Failed to tweet image {frame_filepath} after {retries} attempts')
        # Wait for user input before exiting
        input("Too many errors.")
        sys.exit()
       
    # Save progress to text file
    with open("progress.txt", "w") as f:
        frames_posted += 1
        f.write(str(frames_posted))
    
    print(f"Image posted. Waiting for {wait_time} seconds.\n")
    time.sleep(wait_time)